# SEAS Benchmark Problem 1

Firts of all, run the `bp1.jl` (at the current directory) to simulate the problem defined by **Benchmark Problem 1**. Details are well documented in that. For *3000 years* simulation, it take around *1100 seconds* to complete solving ODEs on *Intel i7 2.9 GHz* by using one core only.

## Retrive our results

In [1]:
import h5py
import numpy as np

/Users/spc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
f = h5py.File('./bp1.h5', 'r')
d = f['bp1']

t = np.array(d['t'])
velocity = np.array(d['velocity'])
state = np.array(d['state'])
shear_stress = np.array(d['shear_stress'])
slip = np.array(d['slip'])

f.close()

## Plot time-series output at specific depth

Use `Plotly` to present our results

In [3]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

In [14]:
def plot_time_series_at_depth(depth_index, filename='bp1_time_series'):
    tr1 = go.Scattergl(x=t, y=np.log10(velocity[:, depth_index]), mode='lines', name='slip rate')
    tr2 = go.Scattergl(x=t, y=np.log10(state[:, depth_index]), mode='lines', name='state variable')
    tr3 = go.Scattergl(x=t, y=shear_stress[:, depth_index], mode='lines', name='shear stress')
    tr4 = go.Scattergl(x=t, y=slip[:, depth_index], mode='lines', name='slip distance')
    
    fig = tools.make_subplots(rows=2, cols=2, shared_xaxes=True)
    
    fig.append_trace(tr1, 1, 1)
    fig.append_trace(tr2, 1, 2)
    fig.append_trace(tr3, 2, 1)
    fig.append_trace(tr4, 2, 2)
    
    fig['layout'].update(title='Time Series at {:.1f} km'.format(depth_index * 0.025))
 
    fig['layout']['xaxis1'].update(title='year')
    fig['layout']['xaxis2'].update(title='year')
    
    fig['layout']['yaxis1'].update(title='log10(m/s)', fixedrange=True)
    fig['layout']['yaxis2'].update(title='log10(s)', fixedrange=True)
    fig['layout']['yaxis3'].update(title='MPa', fixedrange=True)
    fig['layout']['yaxis4'].update(title='m', fixedrange=True)
    
    return py.iplot(fig, filename=filename)

Plot the time-series at the surface, i.e. `depth_index = 0`.

In [15]:
plot_time_series_at_depth(0)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x1,y3 ]  [ (2,2) x2,y4 ]



## Plot slip profile

Identify coseismic/interseismic by comparing maximum velocity along depth with our pre-defined threshold *1e-3 m/s*. The plot for coseismic slips only extend to *0.7* of the whole depth in order not to mess up with interseismic slips.

In [7]:
seis_threshold = 1e-3
bottom_seis = int(0.7 * 1600)
depths = np.arange(0, 40, 0.025)
depths_seis = depths[:bottom_seis]

max_vel = np.max(velocity[1:], axis=1)
index_seis = max_vel > 1e-3

slip_seis = slip[index_seis]
slip_aseis = slip[~index_seis]

In [14]:
def plot_slip_profile(seis_dsmp=150, aseis_dsmp=100, filename='bp1_slip_profile'):
    
    data = []
    for i in np.arange(0, slip_seis.shape[0], seis_dsmp):
        tr = go.Scatter(
            x=slip_seis[i, :bottom_seis], y=depths_seis, mode='lines',
            line=dict(color='rgb(0,176,246)', width=1),
            name='coseismic',
            showlegend=True if i == 0 else False
        )
        data.append(tr)
        
    for i in np.arange(0, slip_aseis.shape[0], aseis_dsmp):
        tr = go.Scatter(
            x=slip_aseis[i], y=depths, mode='lines',
            line=dict(color='rgb(205, 12, 24)', width=1),
            name='interseismic',
            showlegend=True if i == 0 else False
        )
        data.append(tr)
        
    layout = go.Layout(
        xaxis=dict(title='Slip (m)'),
        yaxis=dict(title='Depth (km)', autorange='reversed'),
    )
    
    fig = dict(data=data, layout=layout)
    return py.iplot(fig, filename=filename)

In [16]:
plot_slip_profile()

The draw time for this plot will be slow for all clients.


/Users/spc/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



## More plots

In [13]:
plot_time_series_at_depth(1000, 'more_plot_overwritten')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x1,y3 ]  [ (2,2) x2,y4 ]



In [39]:
def plot_nth_event(depth_index, time_index, title, filename='first_event'):
    ind1, ind2 = time_index
    
    tr1 = go.Scatter(
        x=t[ind1: ind2], y=velocity[ind1: ind2, depth_index], mode='lines', name='slip rate'
    )
    tr2 = go.Scatter(
        x=t[ind1: ind2], y=shear_stress[ind1: ind2, depth_index], mode='lines', name='shear stress'
    )
    
    fig = tools.make_subplots(rows=1, cols=2)
    
    fig.append_trace(tr1, 1, 1)
    fig.append_trace(tr2, 1, 2)
    
    fig['layout'].update(title='{} at {:.1f} km'.format(title, depth_index * 0.025))
 
    fig['layout']['xaxis1'].update(title='year')
    fig['layout']['xaxis2'].update(title='year')
    
    fig['layout']['yaxis1'].update(title='m/s', fixedrange=True)
    fig['layout']['yaxis2'].update(title='MPa', fixedrange=True)

    return py.iplot(fig, filename=filename) 

In [54]:
ind1 = np.argmin(abs(t - 196.2269075))
ind2 = np.argmin(abs(t - 196.2269090))
plot_nth_event(300, [ind1, ind2], 'First event', 'nth_event')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [53]:
ind1 = np.argmin(abs(t - 2948.8025290))
ind2 = np.argmin(abs(t - 2948.8025310))
plot_nth_event(300, [ind1, ind2], 'Last event', 'nth_event')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

